The DCRNN Model is referred by [7]

In [1]:
!git clone https://github.com/liyaguang/DCRNN.git

Cloning into 'DCRNN'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 325 (delta 27), reused 40 (delta 16), pack-reused 272
Receiving objects: 100% (325/325), 127.90 MiB | 28.83 MiB/s, done.
Resolving deltas: 100% (142/142), done.


Data Preparation

In [2]:
%cd /content/DCRNN/

/content/DCRNN


In [3]:
pip install -r requirements.txt

Find, download, and upload the METR-LA and the PEMS-BAY datasets. Download them to your PC first, then upload them onto the session machine to the left. (/content/DCRNN/data/) In my case, I preuploaded the dataset onto Dropbox

In [7]:
!wget https://www.dropbox.com/s/3uto6j6zba4kcje/metr-la.h5 -P /content/DCRNN/data/pems-bay.h5
!wget https://www.dropbox.com/s/akpm0wf7apg6gy0/pems-bay.h5 -P /content/DCRNN/data/metr-la.h5

--2021-12-27 09:19:02--  https://www.dropbox.com/s/3uto6j6zba4kcje/metr-la.h5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/3uto6j6zba4kcje/metr-la.h5 [following]
--2021-12-27 09:19:03--  https://www.dropbox.com/s/raw/3uto6j6zba4kcje/metr-la.h5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc39433eccea7063d8e0e5e2148.dl.dropboxusercontent.com/cd/0/inline/BcowfaWwWKHEOa0VLs9sasYfm4CGkXwlIUQeBj0w4pS1yJNNUt25kDQdbS_w7tKrzqSGRbds0fmwjuG4QkYAE-kl7SpQ7tZFtd-J21oshO-ILmqPzplxVHIowI2E5P9Z9MdWekfmNmqf9Z1XWhduaZwk/file# [following]
--2021-12-27 09:19:03--  https://ucc39433eccea7063d8e0e5e2148.dl.dropboxusercontent.com/cd/0/inline/BcowfaWwWKHEOa0VLs9sasYfm4CGkXwlIUQeBj0w4pS1yJNNUt25kDQdbS_w7tKrzqSGRbds0fmwjuG4QkYAE-kl7SpQ7tZF

In [ ]:
#%mkdir -p data/{METR-LA,PEMS-BAY}

# METR-LA
!python -m scripts.generate_training_data --output_dir=data/METR-LA --traffic_df_filename=data/metr-la.h5

# PEMS-BAY
!python -m scripts.generate_training_data --output_dir=data/PEMS-BAY --traffic_df_filename=data/pems-bay.h5

Generating training data
tcmalloc: large alloc 1361199104 bytes == 0x55a88753c000 @  0x7ff4841fd1e7 0x7ff481d3d46e 0x7ff481d8dc7b 0x7ff481d8dd18 0x7ff481e35010 0x7ff481e3573c 0x7ff481e3585d 0x55a7cefe1749 0x7ff481d7aef7 0x55a7cefdf437 0x55a7cefdf240 0x55a7cf052973 0x55a7cf04d9ee 0x55a7cefe0bda 0x55a7cf04f737 0x55a7cf04dced 0x55a7cef1fe2b 0x7ff481d7aef7 0x55a7cefdf437 0x55a7cefdf240 0x55a7cf052973 0x55a7cf04d9ee 0x55a7cefe0bda 0x55a7cf04f737 0x55a7cf04d9ee 0x55a7cefe0bda 0x55a7cf04f737 0x55a7cefe0afa 0x55a7cf04e915 0x55a7cefe0afa 0x55a7cf04e915
tcmalloc: large alloc 1361199104 bytes == 0x55a8d901c000 @  0x7ff4841fd1e7 0x7ff481d3d46e 0x7ff481d8dc7b 0x7ff481d8dd18 0x7ff481e35010 0x7ff481e3573c 0x7ff481e3585d 0x55a7cefe1749 0x7ff481d7aef7 0x55a7cefdf437 0x55a7cefdf240 0x55a7cf052973 0x55a7cf04d9ee 0x55a7cefe0bda 0x55a7cf04f737 0x55a7cf04dced 0x55a7cef1fe2b 0x7ff481d7aef7 0x55a7cefdf437 0x55a7cefdf240 0x55a7cf052973 0x55a7cf04d9ee 0x55a7cefe0bda 0x55a7cf04f737 0x55a7cf04d9ee 0x55a7cefe0bda 

After the datasets are loaded, construct a graph as an input for the GCN

In [ ]:
!python -m scripts.gen_adj_mx  --sensor_ids_filename=data/sensor_graph/graph_sensor_ids.txt --normalized_k=0.1\
    --output_pkl_filename=data/sensor_graph/adj_mx.pkl

Run the pretrained model

In [ ]:
# METR-LA
!python run_demo.py --config_filename=data/model/pretrained/METR-LA/config.yaml

# PEMS-BAY
!python run_demo.py --config_filename=data/model/pretrained/PEMS-BAY/config.yaml
#The generated prediction of DCRNN is in data/results/dcrnn_predictions.

SyntaxError: ignored

Or retrain the model completely

In [ ]:
# METR-LA
!python dcrnn_train.py --config_filename=data/model/dcrnn_la.yaml

# PEMS-BAY
!python dcrnn_train.py --config_filename=data/model/dcrnn_bay.yaml

Eval Baseline Methods

In [ ]:
# METR-LA
!python -m scripts.eval_baseline_methods --traffic_reading_filename=data/metr-la.h5